In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import ast

In [2]:
def name(liste_films, df):

    df = df.drop(['ordering', 'category', 'job', 'characters'], axis = 1)
    df['nconst'] = df['nconst'] + ', '
    df = df.groupby('tconst').sum()
    df = df.reset_index()

    df = pd.merge(liste_films,
                  df,
                  left_on = 'film_id_out_KNN',
                  right_on = 'tconst',
                  how = 'left')

    df = df.drop('film_id_out_KNN', axis = 1)
    df['nconst'] = df['nconst'].astype(str)
    df['nconst'] = df['nconst'].apply(lambda x : x.split(','))
    df = df.explode('nconst')
    df['tconst'] = df['tconst'] + ','
    df = df.groupby('nconst').sum()
    df = df.reset_index()
    df = df[~(df['nconst']== ' ')]
    df['nconst'] = df['nconst'].astype(str)
    df['tconst'] = df['tconst'].astype(str)
    df['nconst'] = df['nconst'].apply(lambda x : x.replace(' ', ''))
    df['tconst'] = df['tconst'].apply(lambda x : x.replace(' ', ''))

    return df

In [3]:
def acteur_name(df):

    df['tconst_y'] = df['tconst_y'].astype(str)
    df['tconst'] = df['tconst_x'] + df['tconst_y']
    df = df[['nconst', 'primaryName', 'tconst']]
    df['tconst'] = df['tconst'].apply(lambda x : x.replace(' ', '').replace('[', '').replace(']', '').replace("'", ''))
    df['tconst'] = df['tconst'].apply(lambda x : x.split(','))
    df = df.rename({'tconst': 'knownForTitles'}, axis = 1)
    df = df.rename({'nconst': 'acteur'}, axis = 1)

    return df

In [4]:
def realisateur_name(df):
    
    df = df.groupby('nconst').sum()
    df = df.reset_index()
    df = pd.merge(df,
                    df_title_crew,
                    on = 'nconst',
                    how = 'left')

    df = df[~(df['nconst'] == 'nan')]
    df['tconst_x'] = df['tconst_x'].astype(str)
    df['tconst_y'] = df['tconst_y'].astype(str)
    df['tconst'] = df['tconst_x'] + ',' + df['tconst_y']
    df = df.drop(['tconst_y', 'tconst_x'], axis = 1)
    df['tconst'] = df['tconst'].apply(lambda x : x.replace(' ', '').replace('[', '').replace(']', '').replace("'", '').replace(",,", ','))
    df['tconst'] = df['tconst'].astype(str)
    df['tconst'] = df['tconst'].apply(lambda x : x.split(','))
    df['tconst'] = df['tconst'].apply(lambda x : set(x))
    df['tconst'] = df['tconst'].apply(lambda x : list(x))

    df = pd.merge(df,
                    df_name_basics,
                    on = 'nconst',
                    how = 'left')

    df['tconst_x'] = df['tconst_x'].astype(str)
    df['tconst_y'] = df['tconst_y'].astype(str)
    df['tconst'] = df['tconst_x'] + ',' + df['tconst_y']
    df = df.drop(['tconst_y', 'tconst_x'], axis = 1)
    df['tconst'] = df['tconst'].apply(lambda x : x.replace(' ', '').replace('[', '').replace(']', '').replace("'", '').replace(",,", ','))
    df['tconst'] = df['tconst'].astype(str)
    df['tconst'] = df['tconst'].apply(lambda x : x.split(','))
    df['tconst'] = df['tconst'].apply(lambda x : set(x))
    df['tconst'] = df['tconst'].apply(lambda x : list(x))

    df = df.rename({'tconst': 'knownForTitles'}, axis = 1)
    df = df.rename({'nconst': 'realisateurs'}, axis = 1)
    df = df.explode('knownForTitles')
    df = df[~(df['knownForTitles'] == '')]

    return df

IMPORT DES BASES

In [5]:
df_films = pd.read_csv('BD/P2_G5_films.csv.gz', compression = 'gzip')
liste_films = pd.DataFrame(df_films['film_id_out_KNN'])
liste_films = liste_films.drop_duplicates()
df_films = 0
df_title_principals = pd.read_csv('BD/title.principals.tsv.gz', compression = 'gzip', sep = '\t')
df_acteurs = df_title_principals[(df_title_principals['category'] == 'actor') | (df_title_principals['category'] == 'actress') | (df_title_principals['category'] == 'self')]
df_realisateurs = df_title_principals[df_title_principals['category'] == 'director']
df_title_principals = 0
df_title_crew = pd.read_csv('BD/title.crew.tsv.gz', compression = 'gzip', sep = '\t')
df_title_crew = df_title_crew.drop('writers', axis = 1)

df_title_crew = pd.merge(liste_films,
                      df_title_crew,
                      left_on = 'film_id_out_KNN',
                      right_on = 'tconst',
                      how = 'left')
df_title_crew = df_title_crew.drop('film_id_out_KNN', axis = 1)
df_title_crew = df_title_crew.rename({'directors': 'nconst'}, axis = 1)
df_title_crew['nconst'] = df_title_crew['nconst'].astype(str)
df_title_crew['nconst'] = df_title_crew['nconst'].apply(lambda x : x.split(","))
df_title_crew = df_title_crew.explode('nconst')
df_title_crew = df_title_crew[~(df_title_crew['nconst'] == " ")]
df_title_crew['tconst'] = df_title_crew['tconst'] + ', '
df_title_crew = df_title_crew.groupby('nconst').sum()
df_title_crew = df_title_crew.reset_index()

df_name_basics = pd.read_csv('BD/name.basics.tsv.gz', compression = 'gzip', sep = '\t')
df_name_basics = df_name_basics.drop(['birthYear', 'deathYear', 'primaryProfession'], axis = 1)
df_name_basics = df_name_basics.rename({'knownForTitles' : 'tconst'}, axis = 1)
df_name_basics['tconst'] = df_name_basics['tconst'].apply(lambda x : x.split(','))
df_name_basics['nconst'] = df_name_basics['nconst'].astype(str)
df_name_basics['nconst'] = df_name_basics['nconst'].apply(lambda x : x.replace(' ', ''))
df_acteurs = name(liste_films, df_acteurs)
df_realisateurs = name(liste_films, df_realisateurs)
df_acteurs_name = pd.merge(df_acteurs,
                            df_name_basics,
                            on = 'nconst',
                            how = 'left')
df_acteurs = 0

df_acteurs_name = acteur_name(df_acteurs_name)
df_realisateurs_name = realisateur_name(df_realisateurs)

df_acteurs, df_realisateurs, df_name_basics, df_title_crew = 0, 0, 0, 0

df_realisateurs_name.to_csv('P2_G5_realisateurs.csv.gz', index=False, compression='gzip')

df_acteurs_name.to_csv('P2_G5_acteurs.csv.gz', index=False, compression='gzip')
